<a href="https://colab.research.google.com/github/ilopezfr/image-superres-with-autoencoders-tf/blob/master/Image_Super_Resolution_using_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/ilopezfr/image-superres-with-autoencoders

# Image Super Resolution with Auto-encoders
---

In this tutorial you will learn how to train an Auto-encoder to convert low-quality images into high-quality. 




For example, we can take create a high-res image from Hubble telescope (upper) from an original low-res image. 

<img src="img/OE_51_1_011011_f008.png" width="250" height="250" />

## What is a *low quality* image

A *low quality* image is often characterized by a low resolution of the sensor. When a photo is taken by a cheap mobile phone, it often looks pixelated and lacks all sorts of details that make up a rich and detailed image.

<img src="images/pixelatedpics.jpg"/>

Furthermore, an image can be blurry which means that it again lacks high frequency details that are characteristic of high resolution images. An image can become blurry when the shutter speed is too slow and your photo target is moving or if the camera itself is moving and doesn't have any kind of stabilization.

<img src="images/blurry.png"/>

Finally, an image, especially one taken from a low quality camera, will have some added noise. This happens when a camera is set to a high ISO which therefore increases noise signal with the light signal, this means that the camera will capture more light to illuminate the scene, but graininess will appear more distincly. A situation where this happens is in low light conditions.

<img src="images/noise.jpg"/>

### During this DLI, we'll be focusing on the first kind of degradation which is low resolution pixelated images.

## How to improve a *low quality* image ?

Over the years, many methods have been developed. These range from smooth interpolation methods to deconvolution. While giving acceptable results, these methods often rely on some naive assumptions. Sometimes these assumptions are only adapted on a certain type of environment and won't work in a general case.

For example, deconvolution appliyed to telescope imaging knows how the blur is spread exactly (exact Point Spread Function) and has a very good idea of the distribution of the noise. So this eases the problem a lot.

But what if, we could come up with a method that tries to have a more general vision of such problems ?

That's what **AI-based** methods can provide.

Specifically, we'll be introducing Autoencoders that are a solution to this problem based on Artificial Neural Networks.

### Previous non-AI based methods

#### Filters

Many filters exist that can somewhat enhance an image, but the way they enhance the image is fixed, that is, it's done in the same way for any image.

For example, let's take this image :

<img src="images/low_res_no_filter.png" width=250px/>

Now, let's apply a sharpen filter :

<img src="images/low_res_sharpen.png" width=250px/>

As we can see, no detail is created, we just enhance edges so that the image doesn't look blurry/low res anymore.

Let's try to apply cubic interpolation now and compare :

<img src="images/low_res_cubic.png" width=250px/>

As we can see, this kind of interpolation just smoothes out the image which is even worse than a sharpen filter.

#### Deconvolution

[Deconvolution](https://en.wikipedia.org/wiki/Deconvolution) methods often work better on blurry/noisy images and won't be that good in our case, so we won't be delving into them.

**What if we could do better ?** Let's try and beat that.

# Autoencoders
## What's that ?

Autoencoders are a kind of neural network that tries to learn a representation of its input data, but in a space with much smaller dimensionality. This smaller representation is able to learn important features of the input data but in a different space which can be then used to reconstruct the data. If you are familiar with [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis), especially [Kernel PCA](https://en.wikipedia.org/wiki/Kernel_principal_component_analysis), the goal is the same, but the way it is achieved and the quality obtained differ greatly, in favor of autoencoders.

<img src="images/autoencoder_schema.jpg"/>

The fact that they compress data to a smaller feature space, doesn't mean that autoencoders are good at compression (in general). In fact, they're not that good at compression (even compared to JPEG), unless you specifically want to compress a very specific kind of images, let's say trees, or faces or anything specific.

Of course, as autoencoders encode the input data to another space, this means that they are somewhat lossy, as can be seen on the above image (reconstructed input).

A nice property of autoencoders, is that they adapt to the input and ouput they try to match. This means that if a network can match an input noisy image to a noise-free image, the same network could be used to match input low resolution images to high resolution images, the only difference would be that the networks needs to be retrained using a matching dataset.

## What makes up an Autoencoder

An autoencoder is principally made of a few things :

* **An encoder**
* **A loss function**
* **A decoder**

The **encoder** and **decoder** are usually classical neural networks (usually Convolutional Neural Networks, CNNs, in our case).

The **encoder** tries to reduce the dimensionality of the input while the **decoder** tries to recover our image from this new space, which is a lossy process.

The **loss function** is a way of describing a meaningful difference (or distance) between the input and output. This means that, when training, we'll want to have the smallest possible difference between the input and output, so that the network learns to reconstruct the best possible output.

## What kind of data does an Autoencoder manipulate

An autoencoder encodes the data in a smaller feature space, but to find out how to construct our space, the network needs to understand what we want it to do.

In fact an autoencoder is a kind of lossy compression which we take advantage of. And here "*lossy*" doesn't mean "*loss of details*", in fact it just means that we loose information about the original image. We take advantage of that to tell the network to reconstruct the image with more details, which means that we "lost" the low quality image information and reconstructed a new sharper image !

How can an autoencoder learn to do that ? It needs to find a pattern between a pair of different images. This means that you'll need to show it many different images to tell it *"Hey net, that's a low res image and that's what it could look like in high res, you get it ?"*

Of course, you'll need to show it many different pairs of low res/high res  so that the network finds a pattern and tries to add more details to your input images.

We'll see how to put that in practice below, but before we need to create our network.

## Convolutional Neural Network

If you want more details on what are CNNs and what are their layers, you can check out the Appendix notebook which contains an intuitive explanation.

## The encoder

### Let's design an encoder

Designing a neural network is something that needs trial and error to get right.

Here we'll be designing the following encoder :

<img src="images/model_encoder.png"/ width=450px>

**Note**: the layers' dimensions are noted (depth, width, height).

This might look complicated, but let's have a look at it.

The first layer (input_1) takes the input image, here we see that the input image has (3, 256, 256) dimensions. This means that the image is of size 256x256 and has 3 RGB channels (depth).

```Python
n = 256
chan = 3
input_img = Input(shape=(n, n, chan))
```

What's interesting here, is that we want the model to reduce the input into another space that, if we visualize, won't necessarily be meaningful to us but will be an **encoded** representation of the input in a space where it has *meaning* for the neural network. **You should note that the *smaller space is not necessarily smaller than one image*, but it should be smaller than all the images that the network trained on, otherwise it's not a kind of compression anymore because we could store every image losslessly**.

So, to do that, we use what we saw earlier.

Let's first use a convolution :

```Python
l1 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
```

So here, we put 64 different 3x3 filters. This helps us to retrieve many different features from the input image at its original size, before going down in a smaller space.

Let's put another convolution layer to strengthen the features we already saw.

```Python
l2 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)
```

Now, an important step, we begin to descend in a smaller space. We start to slowly downscale the picture by a scale factor of 2.

```Python
l3 = MaxPooling2D(padding='same')(l2)
```

Let's put a dropout layer here, to be sure that our neurons are already starting to generalize better. So we dropout $30\%$ of them.

```Python
l3 = Dropout(0.3)(l3)
```

Let's do the same as above and use some convolution layers to learn new features in our new and smaller space. We'll try to put even more convolutions (128 convolutions of 3x3). This should be able to learn more features.

Why would we want that you say, even though we're in a smaller space ? :O

Well, the space is smaller, so we have less information, then let's try to have even more different convolutions that will try and compensate for the loss of information because we went in a smaller space where we lost some information. This gives us much more *perspective*, especially if you imagine these convolution filters as being a *point of view* looking at the same thing (here, the image). *It's like many people (here, the convolution filters) on a team working together and seeing differently the same problem (the image)* (if the analogy helps)

```Python
l4 = Conv2D(128, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)
```

Again, let's descend and get closer to the space where we think we'll find our encoded representation.

*Let's dive deeper* :

```Python
l6 = MaxPooling2D(padding='same')(l5)
```

Again, let's make sure that every neuron is generalizing and not specializing by droping $50\%$ of them this time.

```Python
l3 = Dropout(0.5)(l3)
```

Let's dive one last time. This should be enough, we don't want to go too deep because this could become inefficient, both in terms of performance (remember that we need to learn all these convolutions filters) and if you dive so deep that what's only left is a very small representation, for example of size 1x1 (1 pixel), you can't really have too many different *point of views* (convolution filters) giving a different perspective on a single point in space because it is what it is, a single and alone point in space...

```Python
encoded = Conv2D(256, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)
```

Finally, we have our compressed representation that we think will be good enough.

Let's just create the encoder's model to be able to train it after. This will enable us to see the **encoded** representation later.

```Python
encoder = Model(input_img, encoded)
```

Run the complete code below, it's the entire encoder. We can see a summary of what our encoder looks like right after.

In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from keras.models import Model
from keras import regularizers

n = 256
chan = 3
input_img = Input(shape=(n, n, chan))

l1 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)
l3 = MaxPooling2D(padding='same')(l2)
l3 = Dropout(0.3)(l3)
l4 = Conv2D(128, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)
l6 = MaxPooling2D(padding='same')(l5)
l3 = Dropout(0.5)(l3)
encoded = Conv2D(256, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)
encoder = Model(input_img, encoded)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [2]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     147584    
__________

As we can see, the network looks like the one we described in the above image. Also, there are a lot of parameters (Trainable params) that will need to be trained. Thankfully, the learning algorithm will do the job for us :)

## The decoder

Let's now create our decoder !

So we want to try doing the steps in **almost reverse order** from our **encoder**.

<img src="images/model_decoder.png"/ width=450px>

We just want to come back from the network's representation space to the real world, one we can hopefully understand.

So, here we take our **encoded** representation in input and just scale it 2x.

```Python
l8 = UpSampling2D()(encoded)
```

We're reversing our network here, so let's put some convolutions filters with the same size as the ones before.

We'll do it in a "symmetric" fashion compared to the encoder.

So here, we want to learn to activate neurons on features which learned what is the difference from a pixelated image to a sharper more detailed image in our current space.

```Python
l9 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l8)
l10 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l9)
```

Here we'll use a merge layer which performs an add operation on the layer just above it and a layer that was way before.
This means that we perform the following operation : convolution2D_4 + convolution2D_7

**Note** : convolution2D_4 and convolution2D_7 are the names on the above image.

**Why** would we want to do that ?

Well, there are a few reasons :

* First, imagine that you want to "share" knowledge from the encoder to the decoder like "*Hey, I'm trying to reconstruct this part of the image, did it roughly look like that to you also ?*".

* It also enables the network to not "loose" information from going deeper. In fact, when a network becomes too "deep", neurons that are at the beginning of the network can start to have difficulty learning because they're too far from the deeper neurons and can't have their knowledge shared. *This called the vanishing gradient problem*, if you want more info, look it up.

Anyway, it's just an addition in the end ;)

```Python
l11 = add([l5, l10])
```

You can guess what's next, we do the usual steps in reverse :

* Come back from the abstract world of the autoencoder (just scale 2x with an Upsampling)
* Tell to the neurons that they need to generalize and not specialize (We dropout $30\%$ of the neurons)
* And again try to find interesting features in our recovered and bigger space (with the convolution filters)

```Python
l12 = UpSampling2D()(l11)
l3 = Dropout(0.3)(l3)
l13 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l12)
l14 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l13)
```

Again, we add the current output from the above layer and a layer that was way before. 

So we just perform : convolution2D_2 + convolution2D_9

```Python
l15 = add([l14, l2])
```

Finally, we want to ouput our final image with the correct number of channels, which is 3 for RGB.

```Python
# chan = 3, for RGB
decoded = Conv2D(chan, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l15)

# Create our network
autoencoder = Model(input_img, decoded)
# You'll understand later what this is
autoencoder_hfenn = Model(input_img, decoded)
```

In [7]:
!wget http://ec2-18-223-16-204.us-east-2.compute.amazonaws.com/lQJp3VdP/tree/data/rez/cars_train
  

--2019-03-25 03:25:52--  http://ec2-18-223-16-204.us-east-2.compute.amazonaws.com/lQJp3VdP/tree/data/rez/cars_train
Resolving ec2-18-223-16-204.us-east-2.compute.amazonaws.com (ec2-18-223-16-204.us-east-2.compute.amazonaws.com)... 18.223.16.204
Connecting to ec2-18-223-16-204.us-east-2.compute.amazonaws.com (ec2-18-223-16-204.us-east-2.compute.amazonaws.com)|18.223.16.204|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15868 (15K) [text/html]
Saving to: ‘cars_train’

cars_train          100%[===================>]  15.50K  --.-KB/s    in 0.07s   

2019-03-25 03:25:52 (226 KB/s) - ‘cars_train’ saved [15868/15868]



## Complete network

So, let's check out our model.

As you can see, it seemed pretty complex, but now that we broke it down, everything seems to make sense. (*If not, I did a bad job :/* )

<img src="images/model.png"/ width=450px>

Here's the final network :

In [0]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from keras.models import Model
from keras import regularizers

# Encoder

n = 256
chan = 3
input_img = Input(shape=(n, n, chan))
l1 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)

l3 = MaxPooling2D(padding='same')(l2)
l3 = Dropout(0.3)(l3)
l4 = Conv2D(128, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)

l6 = MaxPooling2D(padding='same')(l5)
l3 = Dropout(0.5)(l3)
l7 = Conv2D(256, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)

In [0]:
# Decoder

l8 = UpSampling2D()(l7)

l9 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l8)
l10 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l9)

l11 = add([l5, l10])
l12 = UpSampling2D()(l11)
l3 = Dropout(0.3)(l3)
l13 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l12)
l14 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l13)

l15 = add([l14, l2])

# chan = 3, for RGB
decoded = Conv2D(chan, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l15)

# Create our network
autoencoder = Model(input_img, decoded)
# You'll understand later what this is
autoencoder_hfenn = Model(input_img, decoded)

In [5]:
autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_6[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
dropout_3 

The final network has even more parameter to train, so this should take some time to train.

As you will see, training time really depends on the number of parameters that the network has to train. Of course, it also depends on the size of the input data given. That's why the network you design in the future need to have a balance between performance, number of parameters and a reasonable dataset size.

Let's *compile* the model to be able to train it. We'll simply use a Mean Squared Error (MSE) for the loss, we'll see later what this is and if we can go further.

In [0]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

What happens then, is that we have a dataset (a subset of Image Net for speed reasons) of images and that we load it by batches. Why ?

Well, the dataset doesn't really hold in memory, so we split it by batches and give it to the GPU so that it can train on a reasonable part of the dataset at each iteration.

In [0]:
import os
import re
from scipy import ndimage, misc
from skimage.transform import resize, rescale
from matplotlib import pyplot
import numpy as np

def train_batches(just_load_dataset=False):

    batches = 256 # Number of images to have at the same time in a batch

    batch = 0 # Number if images in the current batch (grows over time and then resets for each batch)
    batch_nb = 0 # Batch current index

    max_batches = -1 # If you want to train only on a limited number of images to finish the training even faster.
    
    ep = 4 # Number of epochs

    images = []
    x_train_n = []
    x_train_down = []
    
    x_train_n2 = [] # Resulting high res dataset
    x_train_down2 = [] # Resulting low res dataset
    
    for root, dirnames, filenames in os.walk("/dli/data/rez/cars_train"):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|JPEG|png|bmp|tiff)$", filename):
                if batch_nb == max_batches: # If we limit the number of batches, just return earlier
                    return x_train_n2, x_train_down2
                filepath = os.path.join(root, filename)
                image = pyplot.imread(filepath)
                if len(image.shape) > 2:
                        
                    image_resized = resize(image, (256, 256)) # Resize the image so that every image is the same size
                    x_train_n.append(image_resized) # Add this image to the high res dataset
                    x_train_down.append(rescale(rescale(image_resized, 0.5), 2.0)) # Rescale it 0.5x and 2x so that it is a low res image but still has 256x256 resolution
                    batch += 1
                    if batch == batches:
                        batch_nb += 1

                        x_train_n2 = np.array(x_train_n)
                        x_train_down2 = np.array(x_train_down)
                        
                        if just_load_dataset:
                            return x_train_n2, x_train_down2
                        
                        print('Training batch', batch_nb, '(', batches, ')')

                        autoencoder.fit(x_train_down2, x_train_n2,
                            epochs=ep,
                            batch_size=10,
                            shuffle=True,
                            validation_split=0.15)
                    
                        x_train_n = []
                        x_train_down = []
                    
                        batch = 0

    return x_train_n2, x_train_down2

If you wanted to train the model, that's how you'd do :

```Python
x_train_n = []
x_train_down = []
x_train_n, x_train_down = train_batches()
```

But, we'll just load pretrained model as this takes a lot of time. But first, we'll load the dataset :

In [0]:
x_train_n, x_train_down = train_batches(just_load_dataset=True)

And here, we load the already existing weights :

In [0]:
autoencoder.load_weights("/dli/data/rez/sr.img_net.mse.final_model5.no_patch.weights.best.hdf5")

## Display the results

You need to load the encoder's weight so that you can see the **encoded** representation :

In [0]:
encoder.load_weights('/dli/data/rez/encoder_weights.hdf5')

Here, we feed the encoder the input images without decoding them so that we can vizualise later the result (and hope to understand how our network encoded our data...)

In [0]:
encoded_imgs = encoder.predict(x_train_down)

As we can see, our encoded representation have weird dimensions. We have a representation for each image (256 images), each representation is in 3 dimensions (64x64x256) :

In [0]:
encoded_imgs.shape

After this, we encode in our smaller space and then come back and have a super resolution image :

In [0]:
# We clip the output so that it doesn't produce weird colors
sr1 = np.clip(autoencoder.predict(x_train_down), 0.0, 1.0)

Now run this code below and try to guess which is which. Here are what you shoud guess :

* The low resolution input image
* The reconstructed image
* The original perfect image
* The encoded image
* A bicubic interopolated version

In [0]:
image_index = 251

In [0]:
import matplotlib.pyplot as plt

# Shuffle the pictures so there's more challenge (and you don't cheat :) )
shuffled_array = []
shuffled_array.append((x_train_down[image_index], "none"))
shuffled_array.append((x_train_down[image_index], "bicubic"))
shuffled_array.append((encoded_imgs[image_index].reshape((64*64, 256)), "none"))
shuffled_array.append((sr1[image_index], "none"))
shuffled_array.append((x_train_n[image_index], "none"))
shuffled_array = np.array(shuffled_array)
np.random.shuffle(shuffled_array)

# Display the images
plt.figure(figsize=(128, 128))
i = 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
plt.show()

**Note** : double click on the images to make them bigger

In [0]:
# Type in your guesses

*
*
*
*
*

Did you guess correctly ? We'll see below :)

Finally, we can see our network bear fruits as we look at the images below.

On leftmost side, the input low res image.

On the middle-left, we can see an image upscaled using a bicubic interpolation. The result just looks very smoothed out, no details were really added.

On the middle (weird violet image) we can see our smaller space representation. If we want to vizualize our smaller space that encodes our images in 3D, we need to convert it to 2D. We do that by merging 2 of the 3 dimensions, which will give us something we can look at (you'll need to zoom in to see something). Let's hope we understand how our network thinks.<details> <summary>(**Spoiler, click on this text at your own risk ;)**)</summary>*We won't...*</details>

On the middle-right side, the super resolution recovered image, sharper as expected.

On the rightmost, a perfect version of our input image.

**But**, you can see that MSE is not that good **but we'll see how to improve upon that if you continue reading ;)**.

**Try playing** with the *image_index* variable to see how the model behaves for different images.

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(128, 128))
i = 1
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_down[image_index])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_down[image_index], interpolation="bicubic")
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(encoded_imgs[image_index].reshape((64*64, 256)))
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(sr1[image_index])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(sr1[image_index])
plt.show()

**Note** : double click on the images to make them bigger

###         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    - Legend :  Left (input), Mid-Left (bicubic), Mid (encoding), Mid-Right (output), Right (ground truth)

Now that we have our final network, let's try to enhance it, but first, we need to understand how we measure image quality between images.

## Exercises :

### 1. Try this out yourself and play with the "image_index" variable and see what the network outputs.

### 2. Try to play with the different layers' parameters and see what happens (improvements/worsening). You won't be able to load the already trained weights as the network won't be the same anymore (unless you only change layers that don't need training).

### BONUS 1. Try your own architecture

### BONUS 2. Try to understand the encoded space... Good luck :)

## How to measure image quality

To be able to tell apart two images that have a different visual quality and level of detail, we need to introduces metrics that will help us define our **loss function**.

During this sections, we'll show how we can understand what metrics tell us about quality loss/change between two images.

For this, we'll use this set of images :

In [0]:
image_index = 10

Again, you can play with the "image_index" variable to compare the following metrics on different images

In [0]:
plt.figure(figsize=(128, 128))
i = 1

# Here, we load the images and display them
high_res = x_train_n[image_index]
low_res = x_train_down[image_index]

ax = plt.subplot(10, 10, i)
plt.imshow(high_res)
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(low_res)
plt.show()

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   -Legend: Left (High res), Right (Low res)

### MSE

[Mean Squared Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) is a metric that indicates perfect similarity if the value is 0.

The value grows beyond 0 if the average difference between pixel intensities increases.

MSE has a few issues when used for similarity comparison, meaning that if you take an image and photoshop it to make it brighter and compare it with the original, the two images will be very different according to MSE as a dark image has pixel values closer to 0 and a bright image has pixel closer to 1, this means that the difference is going to be very big as MSE sees the image from a very general point of view.

Anyway, for our purpose, we compare lower quality images to their higher resolution counterpart, so the brightness is retained, this means that ** MSE is still a useful metric in our case**.

In [0]:
def mse(orig, res):
    return ((orig - res) ** 2).mean()

In [0]:
mse(high_res, low_res)

### SSIM

[Structural similarity (SSIM)](https://en.wikipedia.org/wiki/Structural_similarity) measures the similarity between two images as would be perceived on a television or a similar media.

The SSIM is a value in the range $[1, -1]$ where $1$ would mean two indentical images, and lower values would show a "perceptual" difference.

This metric compares small windows in the image rather than the whole image (like MSE), which makes it a bit more interesting.

In [0]:
import skimage.measure
import os
from matplotlib import pyplot

def ssim(ori, res):
    return skimage.measure.compare_ssim(ori.astype(np.float64),
        res.astype(np.float64),
        gaussian_weights=True, data_range=1., win_size=1,
        sigma=1.5, multichannel=False, use_sample_covariance=False)

In [0]:
ssim(high_res, low_res)

### PSNR

[Peak signal-to-noise ratio](https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio) is a metric defined using Mean Squared Error (seen above). 

*PSNR is most commonly used to measure the quality of reconstruction of lossy compression* - Wikipedia

Low resolution and pixelization can be considered as a form of *compression* as we loose information.

When there's no noise, the PSNR is infinite (because there's a division by the MSE and MSE is $0$ when both images are exactly the same).

Of course, this means that we need to maximize the PSNR. The result is in decibel (dB).

In [0]:
def psnr(ori, res):
    return skimage.measure.compare_psnr(ori, res, data_range=1.)

In [0]:
psnr(high_res, low_res)

### HFENN

The HFENN (High Frequency Error Norm Normalized) metric gives  a  measure  of how high-frequency details differ between two images.

This means that we can guess if an image has more or less high frequency details (which are fine details that you need to zoom in to see and that are not blurry) compared to another image.

When the output value is 0 the images are identical. The greater the value, the more of a perceptual difference in both images there is.

In [0]:
import numpy as np
from scipy.ndimage import filters

def l_o_g(img, sigma):
    '''
    Laplacian of Gaussian filter (channel-wise)
    -> img: input image
    -> sigma: gaussian_laplace sigma
    <- filtered image
    '''
    while len(img.shape) < 3:
        img = img[..., np.newaxis]
    out = img.copy()
    for chan in range(img.shape[2]):
        out[..., chan] = filters.gaussian_laplace(img[..., chan], sigma)
    return out

def hfenn(orig, res):
    '''
    High Frequency Error Norm (Normalized) metric for comparison of original and result images
    The metric independent to image size (in contrast to regular HFEN)
    Inputs are expected to be float in range [0, 1] (with possible overflow)
    -> ori: original image
    -> res: result image
    <- HFENN value
    '''
    sgima = 1.5  # From DLMRI paper
    return np.mean((l_o_g(orig - res, sgima)) ** 2) * 1e4  # magnification

In [0]:
hfenn(high_res, low_res)

## Exercises:

### 1. Try this out yourself and play with the "image_index" variable and try to get a sense of what these metrics mean.

### BONUS: Test new metrics that weren't shown here.

## Combining losses

For our current model, we were using the MSE loss which is interesting, but can we do better ?

We want to do super resolution, so let's try using a loss that literally doesn't just tell us that there was a difference between our low res image and high res image, but that also tells us that there was an improvement, for example, in high frequency details.

All the losses have interesting and unique properties. SSIM and RMSE measure image similarity in their own way, PSNR measures the signal reconstruction quality and HFENN measures high frequency details difference. That said, each of these losses bring something interesting to consider over an image.

Then... why not **combine** some of them and see what happens ? :)

### MSE and HFENN

Let's try to see what heppens if we combine our MSE with HFENN.

First, we need to make it so that our HFENN loss is compatible with Tensorflow/Keras, so here's the code :

In [0]:
import scipy.ndimage as nd
import scipy.ndimage.filters as filters

import tensorflow as tf

def hfenn_loss(ori, res):
    '''
    HFENN-based loss
    ori, res - batched images with 3 channels
    See metrics.hfenn
    '''
    fnorm = 0.325 # norm of l_o_g operator, estimated numerically
    sigma = 1.5 # parameter from HFEN metric
    truncate = 4 # default parameter from filters.gaussian_laplace
    wradius = int(truncate * sigma + 0.5)
    eye = np.zeros((2*wradius+1, 2*wradius+1), dtype=np.float32)
    eye[wradius, wradius] = 1.
    ker_mat = filters.gaussian_laplace(eye, sigma)
    with tf.name_scope('hfenn_loss'):
        chan = 3
        ker = tf.constant(np.tile(ker_mat[:, :, None, None], (1, 1, chan, 1)))
        filtered = tf.nn.depthwise_conv2d(ori - res, ker, [1, 1, 1, 1], 'VALID')
        loss = tf.reduce_mean(tf.square(filtered))
        loss = loss / (fnorm**2)
    return loss

Then, we want to combine MSE and HFENN. How would we do that ? :O

Easily by doing a sum, and even better, we can weight the sum like follows :

$MSE + weight * HFENN$, where $weight$ is a real number that could be anything that suits us.

We could choose $weight = 10$ and see what happens. Of course this can be tweaked and this value was chosen empirically.

In [0]:
from keras import losses

def ae_loss(input_img, decoder):
    mse = losses.mean_squared_error(input_img, decoder) # MSE
    weight = 10.0 # weight

    return mse + weight * hfenn_loss(input_img, decoder) # MSE + weight * HFENN

**That's it !**

Now, just compile the model with the new loss :

In [0]:
autoencoder.compile(optimizer='adadelta', loss=ae_loss)

If you wanted to train the model, that's how you'd do :

```Python
x_train_n = []
x_train_down = []
x_train_n, x_train_down = train_batches()
```

But, we'll just load the pretrained weights :

In [0]:
autoencoder_hfenn.load_weights("/dli/data/rez/sr.img_net.mse_hfenn.final_model5_2.no_patch.weights.best.hdf5")

Let's see what the network can do after using our new custom loss :

In [0]:
sr_hfenn = np.clip(autoencoder_hfenn.predict(x_train_down), 0.0, 1.0)

Again, can you try and guess which image is which ?

Here are what you shoud guess :

* The low resolution input image
* The reconstructed image with MSE
* The reconstructed image with our custom MSE + HFENN loss
* The original perfect image
* A bicubic interopolated version

In [0]:
image_index = 99

In [0]:
import matplotlib.pyplot as plt

# Shuffle the pictures so there's more challenge (and you don't cheat :) )
shuffled_array = []
shuffled_array.append((x_train_down[image_index], "none"))
shuffled_array.append((x_train_down[image_index], "bicubic"))
shuffled_array.append((sr1[image_index], "none"))
shuffled_array.append((sr_hfenn[image_index], "none"))
shuffled_array.append((x_train_n[image_index], "none"))
shuffled_array = np.array(shuffled_array)
np.random.shuffle(shuffled_array)

# Display the images
plt.figure(figsize=(128, 128))
i = 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(shuffled_array[i - 1][0], interpolation=shuffled_array[i - 1][1])
plt.show()

**Note** : double click on the images to make them bigger

In [0]:
# Type in your guesses

*
*
*
*
*

Did you guess correctly ? The answer's below.

As we can see below our custom loss makes it so that the image is more detailed. One thing that we can notice though, is that we loose a tiny bit of brightness in the final image.

In [0]:
image_index = 99

In [0]:
plt.figure(figsize=(128, 128))
i = 1
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_down[image_index])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_down[image_index], interpolation="bicubic")
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(sr1[image_index])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(sr_hfenn[image_index])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_n[image_index])
plt.show()

**Note** : double click on the images to make them bigger

###         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    - Legend :  Left (input), Mid-Left (bicubic), Mid (MSE), Mid-Right (MSE + HFENN), Right (Ground truth)

If you look a bit closely and check out the lines and edges, you'll se that they're sharper when using MSE and HFENN compared to MSE alone.

In [0]:
plt.figure(figsize=(128, 128))
j = 6
i = 1
idx_1 = 32*j
idx_2 = 32*(j+1)
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_down[image_index, idx_1:idx_2, idx_1:idx_2])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_down[image_index, idx_1:idx_2, idx_1:idx_2], interpolation="bicubic")
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(sr1[image_index, idx_1:idx_2, idx_1:idx_2])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(sr_hfenn[image_index, idx_1:idx_2, idx_1:idx_2])
i += 1
ax = plt.subplot(10, 10, i)
plt.imshow(x_train_n[image_index, idx_1:idx_2, idx_1:idx_2])
plt.show()

**Note** : double click on the images to make them bigger

###         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    - Legend :  Left (input), Mid-Left (bicubic), Mid (MSE), Mid-Right (MSE + HFENN), Right (Ground truth)

## Exercises :

### 1. Try to use a different combination of loss functions and make your own loss.

# What to remember

In the end, we were able to harness the power of deep learning and autoencoders to quickly enhance images.

What you should remember is :

* Autoencoders are deep neural networks
* Autoencoders encode your data into a smaller space
* Autoencoders find patterns between pairs of images and that is what the encoding represents
* The architecture a network is made very empirically
* Your loss and the metrics used are very important and influence the way your network learns

# Appendix

There is another notebook containing more explanations, a "Going further section" and bonus exercises if you want to go further.

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>